# Hierachy Simple PSO Algorithm (THSPSO and PHSPSO)
This Algorithm is from the research paper
* Hao-Ran Liu, Jing-Chuang Cui, Ze-Dan Lu, Da-Yan Liu, Yu-Jing Deng, A hierarchical simple particle swarm optimization with mean dimensional information Applied Soft Computing Journal 76 (2019) 712–725
* In the algorithms, the velocity is eliminated and position update is done at any time using one of the following equations depending on the strategy employed.
* *1. $ X_{id}(t+1) = wX_{id}(t) + c_1*r_1( P_{id}(t)-X_{id}(t) )  $
* *2. $ X_{id}(t+1) = wX_{id}(t)  + c_2*r_2( P_{gd}(t)-X_{id}(t) ) $
* *3. $ X_{id}(t+1) = wX_{id}(t) +  c_3*r_3( P_{ad}(t)-X_{id}(t) ) $


In [147]:
#------------------------------------------------------------------------------+
#   CHIA E TUNGOM (Chemago) {chemago99@yahoo.com}
#   Simpler Particle Swarm Optimization (PSO) Algorithm in Python
#   Feb, 2020
#   [Ref Paper] Hao-Ran Liu, Jing-Chuang Cui, Ze-Dan Lu, Da-Yan Liu, Yu-Jing Deng, A hierarchical simple 
#   particle swarm optimization with mean dimensional information Applied Soft Computing Journal 76 (2019) 712–725
#------------------------------------------------------------------------------+

In [148]:
# Import dependensies

import random
import math

In [180]:
# optimization function
def sphere(x):
    ans = 0
    for i in range(len(x)):
        ans += x[i]**2
        
    return ans

# Population Initialization
def InitializeX(population, dimensions, boundary = (-10, 10)):
    
    bounds = [boundary for i in range(dimensions)]
    return [[random.uniform(bounds[i][0], bounds[i][1]) for i in range(dimensions)] for i in range(population)]


In [181]:
# Calculate global best
def Gbest(f, X):
    
    fX = [f(i) for i in X]
    ind = fX.index(min(fX))
    gbest = X[ind]

    return gbest

# calculate personal Best
def Pbest(f, X, XP): 
    
    """ XP : previous X
        X  : current X  """
    
    fX = [f(i) for i in X]
    fP = [f(i) for i in XP]
    pbest = []
    
    for i in range(len(X)):
        
        if fX[i] < fP[i]: 
            pbest.append(X[i])
        else:
            pbest.append(XP[i])
    
    return pbest

In [182]:
# Cognitive only update 
def COupdateX(X, pbest, bound, w = 0.6, c1 = 2.0):
    
    """pbest: personal best X """
    
    NewX = []
    
    for i in range(len(X)):
        NewX.append([])
        
        for j in range(len(X[0])):
            
            r1 = random.random()
            # Update position
            NewX[i].append( (w*X[i][j]) + (c1 * r1*(pbest[i][j] - X[i][j])) )
            
            # contain particles in bound
            if NewX[i][j] < bound[j][0] or NewX[i][j] > bound[j][1]:
                NewX[i][j] = random.uniform(bound[j][0],bound[j][1])
    return NewX

# Social Only update
def SOupdateX(X, gbest, bound, w = 0.6, c2 = 2.0):
    
    """ gbest: global best """
    
    NewX = []
    
    for i in range(len(X)):
        NewX.append([])
        
        for j in range(len(X[0])):
            
            r2 = random.random()
            # Update position
            NewX[i].append( (w*X[i][j]) + (c2* r2* (gbest[j] - X[i][j])) )
            
            # contain particles in bound
            if NewX[i][j] < bound[j][0] or NewX[i][j] > bound[j][1]:
                NewX[i][j] = random.uniform(bound[j][0],bound[j][1])
    return NewX

# Mean Dimensional update 
def MDupdateX(X, bound, w = 0.6, c3 = 2.0):
    
    # use X to calculate the mead diension
    Md = []
    for i in range(len(X[0])): #dimension length
        #for j in range(len(X)): # population
        Md.append(sum([X[j][i] for j in range(len(X))])/len(X))

    NewX = []
    
    for i in range(len(X)):
        NewX.append([])
        
        for j in range(len(X[0])):
            
            r3 = random.random()
            # Update position
            NewX[i].append( (w*X[i][j]) + (c3* r3* (Md[j] - X[i][j])) )
            
            # contain particles in bound
            if NewX[i][j] < bound[j][0] or NewX[i][j] > bound[j][1]:
                NewX[i][j] = random.uniform(bound[j][0],bound[j][1])
    return NewX

In [183]:
# Write the PSO Loop
# input: dim, pop, bounds, function
dim = 3                                                 # dimension of vector
pop = 5
bound = (-10, 10)                                       # (lower, upper) bound for one dimensional vector
#bounds = [(-10, 10) for i in range(dim)]               # for a 3 dimensionsal vector

def SPSO(dim, bound, function, pop = 10, runs = 20):
    
    X = InitializeX(pop, dim, bound)
    bounds = [bound for i in range(dim)]
    c1 = 2.0
    c2 = 2.0
    c3 = 2.0
    w = 0.9
    
    i = 0
    while i < runs :
        
        gbest = Gbest(function, X)
        if i == 0:
            pbest = X
        else:
            pbest = Pbest(function, X, XP)
            
        XP = X.copy()    
        X = COupdateX(X, pbest, bounds, w, c1) 
        #X = SOupdateX(X, gbest, bounds, w, c2)
        #X = MDupdateX(X, bounds, w, c3)
        
        i += 1
        
        #print(min ([function(i) for i in X]))
    lo = [function(i) for i in X]
    o = min(lo)
    ind = lo.index(o)
    #print(X[ind])
    return min([function(i) for i in X])

In [184]:
dimensions = 50
bound = (-10,10)

SPSO(dimensions, bound, sphere, pop = 20, runs= 50)

0.03550842886973861

In [185]:
import math

def F1(x):
    """ x is a vector of dimension d 
        e.g if x = [1,2,3] , d = 3 or len(x)"""
    
    a = 0
    for i in range(len(x)):
        a += ( (i+1) * ( ( ((2*x[i]) ** 2) - x[i] - 1) **2) )
        
    return ((x[0]+1)**2) + a 

dimensions = 5
bound = (-10,10)

SPSO(dimensions, bound, F1, pop = 50, runs= 1000)


15.999999999999998

In [186]:
def F6(x):
    
    a = math.sqrt( ((len(x))**-1) * sum([i**2 for i in x]) )
    b = math.e ** ( ((len(x))**-1) * sum([math.cos(2*math.pi*i) for i in x]) )
            
    ans  = (-20*( math.e ** (-0.02 * a) ) ) - b + 20 + math.e
    
    return ans

dimensions = 5
bound = (-32,32)

SPSO(dimensions, bound, F6, pop = 50, runs= 1000)


4.440892098500626e-16

# Time Hierachy Strategy (THSPSO)

In this Strategy, the Social Model is used in stage 1, Cognitive model in stage 2 and MD model in stage 3. Recommended values for t1 and t2 are 1/5 and 4/5 respectively based on the research paper which correspond to the times in which a model is used. 

In [187]:
def THSPSO(dim, bound, function, pop = 10, runs = 20):
    
    X = InitializeX(pop, dim, bound)
    bounds = [bound for i in range(dim)]
    c1 = 2.0
    c2 = 2.0
    c3 = 2.0
    w = 0.9
    
    # Time hierachy variables
    t1 = 1/5
    t2 = 4/5
    
    i = 0
    while i < runs :
        
        gbest = Gbest(function, X)
        if i == 0:
            pbest = X
        else:
            pbest = Pbest(function, X, XP)
            
        XP = X.copy()
        
        # Time hierachy loop
        if i/runs < t1:
            X = SOupdateX(X, gbest, bounds, w, c2)
        elif i/runs > t1 and i/runs < t2:
            X = COupdateX(X, pbest, bounds, w, c1)
        else:
            X = MDupdateX(X, bounds, w, c3)
        
        i += 1
        
        #print(min ([function(i) for i in X]))
        
    return min([function(i) for i in X])

In [188]:
dimensions = 50
bound = (-10,10)

THSPSO(dimensions, bound, sphere, pop = 20, runs= 50)

0.01999992844264397

In [189]:
import math

def F1(x):
    """ x is a vector of dimension d 
        e.g if x = [1,2,3] , d = 3 or len(x)"""
    
    a = 0
    for i in range(len(x)):
        a += ( i * ( ((2*x[i]) ** 2) - x[i] - 1) )**2
        
    return (x[0]**2) + a 

dimensions = 5
bound = (-10,10)

THSPSO(dimensions, bound, F1, pop = 50, runs= 1000)


30.0

In [190]:
def F6(x):
    
    a = math.sqrt( ((len(x))**-1) * sum([i**2 for i in x]) )
    b = math.e ** ( ((len(x))**-1) * sum([math.cos(2*math.pi*i) for i in x]) )
            
    ans  = (-20*( math.e ** (-0.02 * a) ) ) - b + 20 + math.e
    
    return ans

dimensions = 5
bound = (-32,32)

THSPSO(dimensions, bound, F6, pop = 50, runs= 1000)


4.440892098500626e-16

# Probability Hierachy Strategy (PHSPSO)
In this strategy, 2 constants pc1 and pc2 in the range 0 and 1 are set. A random uniform number r in the range 0 and 1 is generated. these three variables are then used to chose between CO, SO and MD model during the search. Recommended values for pc1 and pc2 are 0.6 and 0.9 respectively based on the research paper. 

In [191]:
def PHSPSO(dim, bound, function, pop = 10, runs = 20):
    
    X = InitializeX(pop, dim, bound)
    bounds = [bound for i in range(dim)]
    c1 = 2.0
    c2 = 2.0
    c3 = 2.0
    w = 0.9
    
    # set probability hierachy variables
    pc1 = 0.6
    pc2 = 0.9
    
    i = 0
    while i < runs :
        
        gbest = Gbest(function, X)
        if i == 0:
            pbest = X
        else:
            pbest = Pbest(function, X, XP)
            
        XP = X.copy()
        
        # Probability hierachy loop
        
        r = random.random()
        if r < pc1:
            X = SOupdateX(X, gbest, bounds, w, c2)
        elif r > pc1 and r < pc2:
            X = COupdateX(X, pbest, bounds, w, c1)
        else:
            X = MDupdateX(X, bounds, w, c3)
        
        i += 1
        
        #print(min ([function(i) for i in X]))
        
    return min([function(i) for i in X])

In [192]:
dimensions = 50
bound = (-10,10)

PHSPSO(dimensions, bound, sphere, pop = 20, runs= 50)

0.008682526235787993

In [193]:
import math

def F1(x):
    """ x is a vector of dimension d 
        e.g if x = [1,2,3] , d = 3 or len(x)"""
    
    a = 0
    for i in range(len(x)):
        a += ( i * ( ((2*x[i]) ** 2) - x[i] - 1) )**2
        
    return (x[0]**2) + a 

dimensions = 5
bound = (-10,10)

PHSPSO(dimensions, bound, F1, pop = 50, runs= 1000)


14.035538691430139

In [194]:
def F6(x):
    
    a = math.sqrt( ((len(x))**-1) * sum([i**2 for i in x]) )
    b = math.e ** ( ((len(x))**-1) * sum([math.cos(2*math.pi*i) for i in x]) )
            
    ans  = (-20*( math.e ** (-0.02 * a) ) ) - b + 20 + math.e
    
    return ans

dimensions = 5
bound = (-32,32)

PHSPSO(dimensions, bound, F6, pop = 50, runs= 1000)


4.440892098500626e-16

# Test On BenchMark Functions 
Test dimesion is set to 30D, with population size of 40 and maximum of 1000 iterations 

In [195]:
import math

def schewefels(x, n = 4):
    ans = 0
    for i in range(n):
        sphere(x)
        ans += ans 
    return ans 

def rosenbrock(x):
    ans  = 0
    for i in range(len(x)-1):
        ans +=( 100 * (( x[i+1] - (x[i]**2) )**2) + ((x[i]-1)**2) )
    return ans
        
def rastrigen(x):
    ans  = 0
    for i in range (len(x)):
        ans += x[i]**2 - 10*( math.cos(2*math.pi*x[i]) ) + 10
    return ans   

def griewank(x):
    ans = 0
    a = 0
    b = 1
    for i in range(len(x)):
        a += x[i]**2
        b *= ( math.cos( x[i]) / (math.sqrt(i+1) ) )
    ans = ( (1/4000)*a ) + b + 1
    return ans 

def aukley(x):
    a = 0
    b = 0
    ans = 0
    for i in range(len(x)):
        a += x[i]**2
        b += math.cos( 2*math.pi*x[i])
    ans = (-20*math.exp((-0.2)*(math.sqrt( (1/len(x)) *a)) ) )- math.exp( (1/len(x)) * b ) + 20 + math.e
    return ans


def schewefels2(x):
    ans = 0
    a = 0
    b = 1
    for i in range(len(x)):
        a += x[i]
        b *= x[i]
    ans = a + b 
    return ans 

In [196]:
dimensions = 10
bound = (-100, 100)

PHSPSO(dimensions, bound, sphere, pop = 40, runs= 1000), THSPSO(dimensions, bound, sphere, pop = 40, runs= 1000)

(4.144844476155799e-94, 2.1950016769749666e-92)

In [197]:
dimensions = 10
bound = (-100, 100)

PHSPSO(dimensions, bound, schewefels, pop = 40, runs= 1000), THSPSO(dimensions, bound, schewefels, pop = 40, runs= 1000)

(0, 0)

In [198]:
dimensions = 10
bound = (-100, 100)

PHSPSO(dimensions, bound, rosenbrock, pop = 40, runs= 1000), THSPSO(dimensions, bound, rosenbrock, pop = 40, runs= 1000)

(8.77111090273173, 9.0)

In [199]:
dimensions = 10
bound = (-100, 100)

PHSPSO(dimensions, bound, rastrigen, pop = 40, runs= 1000), THSPSO(dimensions, bound, rastrigen, pop = 40, runs= 1000)

(0.0, 0.0)

In [203]:
dimensions = 50
bound = (-100, 100)

PHSPSO(dimensions, bound, griewank, pop = 40, runs= 1000), THSPSO(dimensions, bound, griewank, pop = 40, runs= 1000)

(1.0, 1.0)

In [201]:
dimensions = 50
bound = (-100, 100)

PHSPSO(dimensions, bound, aukley, pop = 40, runs= 1000), THSPSO(dimensions, bound, aukley, pop = 40, runs= 1000)

(4.440892098500626e-16, 4.440892098500626e-16)

In [202]:
dimensions = 50
bound = (-100, 100)

PHSPSO(dimensions, bound, schewefels2, pop = 40, runs= 1000), THSPSO(dimensions, bound, schewefels2, pop = 40, runs= 1000)

(-99.60348471032269, -2.742772913230144e-19)